In [1]:
import torch.nn.functional as F
from torchvision import transforms as transform_lib
from torchvision.datasets import MNIST
from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule
%load_ext autoreload
%autoreload 2

In [ ]:
from dvrl.training.experiments import RemovingSamplesExperiment
from dvrl.training.train import run_mnist_dvrl

In [ ]:
prediction_hparams = {'pred_hidden_dim': 256,
                      'pred_num_layers': 5,
                      'activation_fn': F.relu,
                      'predictor_lr': 1e-3,
                      'num_classes': 10,
                      'inner_batch_size': 32,
                      'num_inner_iterations': 100
                      }
dvrl_hparams = {'dve_hidden_dim': 256,
                'dve_comb_dim': 128,
                'dve_num_layers': 5,
                'dve_lr': 1e-2,
                'epsilon': 1e-8,
                'T': 20,
                'num_classes':10,
                'outer_batch_size':2000
                }

In [ ]:
pred_model, dve_model = run_mnist_dvrl(prediction_hparams, dvrl_hparams)


In [ ]:
DATA_PATH = 'data/raw'
datamodule = MNISTDataModule(data_dir=DATA_PATH + '/mnist_')
datamodule.prepare_data()  # downloads data to given path
train_dataloader = datamodule.train_dataloader(batch_size=dvrl_hparams.get('outer_batch_size', 32))
val_dataloader = datamodule.val_dataloader(batch_size=dvrl_hparams.get('outer_batch_size', 32))
test_dataloader = datamodule.test_dataloader(batch_size=dvrl_hparams.get('outer_batch_size', 32))

In [ ]:
transforms = transform_lib.Compose([transform_lib.ToTensor()])
dataset = MNIST(DATA_PATH + '/mnist_', train=True, download=False, transform=transforms)

exp_remove_samples = RemovingSamplesExperiment(dataset, dve_model, train_dataloader, val_dataloader, test_dataloader,
                                               prediction_hparams)
exp_remove_samples.run_experiment()